In [1]:
from random import shuffle
import glob
import cv2
import matplotlib.pyplot as plt
import re
import numpy as np
import itertools
from scipy.spatial import distance
%matplotlib inline

In [2]:
## loads the points of a sphere to make assign groups in the classification process
reference_points = np.genfromtxt('points_in_sphere.txt')

In [ ]:
def config_values(dir_file):
    with open(dir_file, 'r') as config_folder:
        for line in itertools.islice(config_folder, 12):pass
        val_octree = re.findall(r'[+-]?[0-9.]+', line)
        #print(float(val_octree[1]))
    return float(val_octree[1])

In [ ]:
dataset_dirs = [1,2,3,4,5,6,7,8,9,10,12,14]
volumes = []
clase = []
pose = []
orn = []
#This programm labels the positions for every pose in the octree files
for _, fig in enumerate(dataset_dirs):
    if (fig > 6):
        folder = sorted(glob.glob('/home/miguelmg/Documents/CIDETEC/semestre 2/vision 3d/proyecto/6d pose/hinterstoisser/nubes/modelo' + str(int(fig)) + '/nbv/*'))
    else:
        folder = sorted(glob.glob('/media/miguelmg/EC6A24366A240046/dataset/modelo' + str(int(fig)) + '/nbv/*'))
    numpy_file = "/home/miguelmg/Documents/CIDETEC/semestre 2/vision 3d/proyecto/6d pose/hinterstoisser/nubes/dataset/"
    print('\n Processing folder: ', fig, ' :',end = ' ')

    ##octree configuration parameters 
    min_octrees_file = config_values('/home/miguelmg/repositorios/vpl/data_example/FreeFlyer/config/partialModelConfig_model' + str(int(fig)) + '.ini')
    max_octrees_file = -min_octrees_file
    min_cubo = 0
    max_cubo = 32

    m = (max_cubo - min_cubo) / (max_octrees_file - min_octrees_file)
    output_cube_size = 32*32*32

    ## start the iterations over all the folders contained in the input folder
    pos_indice = 0
    folder_indice = 0
    dataset = []
    i_modulo = 0
    
    for folders in folder:
        number = re.search('modelo' + str(int(fig)) + '/nbv/(.+)', folders)
        #print(number.group(1))
        #print (folders)
        sub_folder = sorted(glob.glob(folders + '/octo_acum/*'))
        num_files =	int(len(sub_folder)/3) 
        ## all the octrees files are examined
        for file_ in range(num_files):
            file_ = file_ + 1

            ### processing steps for the octree files
            octree_file = np.loadtxt(folders + '/octo_acum/octomap_acum_' + str(file_) + '.txt', dtype = float) ## the current octree file is loaded in octree_file
            #print(folders + '/octo_acum/octomap_acum_' + str(file_) + '.txt')
            x = octree_file[:,0]
            y = octree_file[:,1]
            z = octree_file[:,2]
            v = octree_file[:,3]
            output_cube = np.zeros((32,32,32))
            ##### the cube of the octree of size 32x32x32 is obtained
            for i in range(output_cube_size):
                x_cube = int((x[i]*m*2+32)/2)
                y_cube = int((y[i]*m*2+32)/2)
                z_cube = int((z[i]*m*2+32)/2)
                output_cube[x_cube][y_cube][z_cube] = v[i]
            ##### flatten representation of the cube is obtained
            flatten_cube = output_cube.reshape(32*32*32)
            ##### the first term of the tuple is saved in tuple_first
            #tuple_first = output_cube

            ### processing steps for the position, one hot enconding is obtained
            num_point = 0
            min_distance = 1
            pos_file = np.genfromtxt(folders + "/poses/pose_orientation/pose_orn" + str(file_) + ".dat")
            pos_coord = pos_file[0]
            orn_coord = pos_file[1]
            #print(orn_coord)
            ###################################################
            ############# CLASE ###############################
            ###################################################
            ###### the current pose is compared with all the points in the sphere
            for points in reference_points:
                distance_ = distance.euclidean(points, pos_coord)
                if (distance_ < min_distance):
                    min_distance = distance_
                    pos_indice = num_point
                num_point = num_point + 1
            ######### one hot encoding is created
            clase.append([pos_indice])
            ####################################################
            
            
            ####################################################
            ################ Regression ########################
            ####################################################
            pose.append(pos_coord)
            orn.append(orn_coord)
            
            ####################################################
            ################ Volumes ###########################
            ####################################################
            volumes.append(flatten_cube)
        if (i_modulo % 50 ==0):
            print(number.group(1) , ' ,', end = ' ' )
        i_modulo += 5
        

pose = np.array(pose)
orn = np.array(orn)
clase = np.array(clase)
volumes = np.array(volumes)
        
#np.save(numpy_file + 'dataset_volmues', volumes)
#np.save(numpy_file + 'dataset_lbl_pose', pose)
#np.save(numpy_file + 'dataset_lbl_orn', orn)
#np.save(numpy_file + 'dataset_lbl_clase', clase)



 Processing folder:  1  : 0  , 1035  , 1080  , 1125  , 1170  , 1215  , 1260  , 1305  , 170  , 215  , 260  , 305  , 350  , 40  , 445  , 490  , 530  , 575  , 620  , 665  , 710  , 755  , 800  , 85  , 895  , 940  , 985  , 
 Processing folder:  2  : 0  , 1035  , 1080  , 1125  , 1170  , 1215  , 1260  , 1305  , 170  , 215  , 260  , 305  , 350  , 40  , 445  , 490  , 530  , 575  , 620  , 665  , 710  , 755  , 800  , 85  , 895  , 940  , 985  , 
 Processing folder:  3  : 0  , 1035  , 1080  , 1125  , 1170  , 1215  , 1260  , 1305  , 170  , 215  , 260  , 305  , 

In [ ]:
pose = np.array(pose)
clase = np.array(clase)
volumes = np.array(volumes)
print((volumes.shape))

### Pruebas para cargar dataset

In [ ]:
dataset_load_path = "/home/miguelmg/Documents/CIDETEC/semestre 2/vision 3d/proyecto/11_ref_100_vol.npy"
dataset_load_lbl_path = "/home/miguelmg/Documents/CIDETEC/semestre 2/vision 3d/proyecto/11_ref_100_lbl.npy"
dataset = np.load(dataset_load_path)
print(dataset.shape)

In [ ]:
labels_ = np.load(dataset_load_lbl_path)
print((labels_.shape))
labels_ = np.array(labels_)
print(labels_)

In [ ]:
one_hot = np.zeros((len(labels_),14))
one_hot[np.arange(len(labels_)),labels_]=1
print(one_hot)